In [ ]:
import polars as pl
#import itables.options as opt
from decouple import config
#from itables import init_notebook_mode

#init_notebook_mode(all_interactive=True)
#opt.maxBytes = 0

userpg = config('POSTGRES_USER')
passpg = config('POSTGRES_PASS')
hostpg = config('POSTGRES_HOST')
portdbpg = config('POSTGRES_PORT')
dbnamepg = config('POSTGRES_DB')
usermsql = config('MS_USER')
passmsql = config('MS_PASS')
hostmsql = config('MS_HOST')
portmsql = config('MS_PORT')
dbnamemsql = config('MS_DB')

In [ ]:
uri = f'postgresql://{userpg}:{passpg}@{hostpg}:{portdbpg}/{dbnamepg}'
query = """
        select * from "hq_dea_3a9df112-2351-459e-97a6-468d1cfaaf91"."Complemento$2"
"""
df = pl.DataFrame(pl.read_database_uri(query=query, uri=uri, engine='connectorx'))
df

In [ ]:
df.columns
#seleccionar solo columnas especificas
df = df.select(['interview__id','fecha','depto','municipio'])
#renombrar una columna primero valor original luego nuevo nombre columna 
df = df.rename({'interview__id':'llave'})
df


In [ ]:
#creo un nuevo df llamado dfproductor donde selecciono solo ciertas columnas 
dfProductor = df.select(['llave', 'fecha', 'depto', 'municipio', 'cton','caserio','rubros','tipo_informante',
                         'nombre','direccion','telefono', 'lugar_entrevista'])
print(dfProductor)
# codigo para saber la cantidad de filas y columnas, el primer valor son las filas el segundo las columnas del df
rows = dfProductor.shape
print(rows)

In [ ]:
query = """
        select * from ws_dea.reusablecategoricaloptions 
        where questionnaireid = 'a1b0a945-f180-4379-bd35-2faae3f2b739'
    """
dfcatalogo = pl.DataFrame(pl.read_database_uri(query=query, uri=uri, engine='connectorx'))
dfdepto = dfcatalogo.filter((pl.col('categoriesid')=='4c204179-60b5-cbcb-a424-5ddb75b9a290')&(pl.col('questionnaireversion')==2)).rename({'text':'Ndepartamento',
                            'value':'iddepto'}).select(['iddepto', 'Ndepartamento'])
dfmunicipio = dfcatalogo.filter((pl.col('categoriesid')=='b55c5b44-8b65-65ff-7a72-962d975e3fd2')&(pl.col('questionnaireversion')==2)).rename({'value':'idmuni'
                                ,'text':'Nmunicipio','parentvalue':'iddepto'}).select(['idmuni','Nmunicipio','iddepto'])
dfcanton = dfcatalogo.filter((pl.col('categoriesid')=='346c75ec-2fff-3acc-ec8f-eba7e234b83e')&(pl.col('questionnaireversion')==2)).rename({'value':'idcanton',
                             'text':'Ncanton','parentvalue':'idmuni'}).select(['idcanton','Ncanton', 'idmuni'])
dffrutas = dfcatalogo.filter((pl.col('categoriesid')=='08f20ae9-440c-2054-0815-d28835790226')).select(['value','text']).rename({'value':'idfruta',
                             'text':'Nfruta'})
dfhortalizas = dfcatalogo.filter((pl.col('categoriesid')=='9700cafd-5a82-cad1-354f-2898a4b7bad8')).select(['value','text']).rename({'value':
                                  'idhortaliza','text':'Nhortaliza'})
dfagroindust = dfcatalogo.filter((pl.col('categoriesid')=='94fe2c9e-f86c-4d8c-9c25-10c3afc807a4')&(pl.col('questionnaireversion')==2)).select(
                                 ['value','text']).rename({'value':'idagro','text':'agroindustrial'})
dffrutas

In [ ]:
dfprod = dfProductor.join(dfdepto, left_on='depto', right_on='iddepto', how='inner').join(dfmunicipio, left_on='municipio', right_on='idmuni', 
                          how='inner').join(dfcanton, left_on='cton', right_on='idcanton', how='inner').select(['llave','fecha','Ndepartamento'
                          ,'Nmunicipio','Ncanton','caserio','rubros','tipo_informante','nombre', 'direccion','telefono', 'lugar_entrevista'])
#Ordenar por un o varios campos, unique funciona para eliminar duplicados especificando el campo, keep para especificar que duplicado mantener
dfprod.sort('Ndepartamento','Nmunicipio').unique('Ndepartamento',keep = 'last', maintain_order= True)
#dividir una columna por un caracter separador 
dfprod.with_columns(
    #especificar el nombre de la columna 
    pl.col('Ncanton')
    #especificar el caracter separador y cuantas veces lo debe separar
    .str.split_exact(' ',1)
    #renombrando los nuevos campos que se generaron del split
    .struct.rename_fields(['test1','test2'])
    #creo un nuevo campo de tipo struct donde ya esta separada mi columna
    .alias('split canton')
).unnest('split canton')#unnest me divide en columna los resultados en mi columna split canton

#eliminar espacios en blanco al inicio o final del texto o un caracter especifico
dfprod.with_columns(
       pl.col('Ncanton').str.strip_chars().alias('espacioEnBlanco')
).select(['llave','Ndepartamento','Nmunicipio','Ncanton','espacioEnBlanco'])
#Reemplazar un o varios caracteres o palabras, ascii_case_insensitive para que no sea sensible a mayusculas y minusculas
dfprod.with_columns(
    pl.col('Ncanton').str
    .replace_many(['a'],' ', ascii_case_insensitive=True)
    .alias('reemplazar')
)

In [ ]:
query = """
            select * from "hq_dea_3a9df112-2351-459e-97a6-468d1cfaaf91"."Complemento$1_Cultivo_Frutas" 
"""
dfprodfrutas = pl.DataFrame(pl.read_database_uri(query=query, uri=uri, engine='connectorx')).select(['interview__id','roster__vector','epoca_frutas','areaplantillaf',
                    'areaprodf','produccionf','observacionf']).rename({'interview__id':'llave','roster__vector':'cultivo','epoca_frutas':'epoca'}).explode('cultivo').cast({'cultivo':pl.Int32})
dfprodfrutas = dfprodfrutas.join(dffrutas, left_on='cultivo', right_on='idfruta', how='inner').select(['llave','Nfruta','epoca','areaplantillaf'
                                ,'areaprodf','produccionf','observacionf'])
dfprodfrutas
query = """
            select * from "hq_dea_3a9df112-2351-459e-97a6-468d1cfaaf91"."Complemento$1_Hortaliza_Cultivada"
"""
dfprodhortalizas = pl.DataFrame(pl.read_database_uri(query=query, uri=uri, engine='connectorx')).explode('roster__vector').cast({'roster__vector':
                                pl.Int32}).rename({'interview__id':'llave','roster__vector':'cultivo'}).select(['llave','cultivo','epoca','estructura'
                                ,'tipo_estructura','gps_estructura','area_produccion','produccion','observacion_horta'])
dfprodhortalizas = dfprodhortalizas.join(dfhortalizas, left_on='cultivo', right_on='idhortaliza', how='inner').select(['llave','Nhortaliza',
                                         'epoca','estructura','tipo_estructura','gps_estructura','area_produccion','produccion','observacion_horta'])
query = """
            select * from "hq_dea_3a9df112-2351-459e-97a6-468d1cfaaf91"."Complemento$1_Cultivo_AGRO"
"""
dfprodagroindustriales = pl.DataFrame(pl.read_database_uri(query=query, uri=uri, engine='connectorx')).explode('roster__vector').cast({'roster__vector':
                                       pl.Int32}).rename({'interview__id':'llave','roster__vector':'cultivo'}).select(['llave','cultivo','epoca_agro',
                                      'aplantillagro','areaprodagro','produccionagro','observacionagro'])
dfprodagroindustriales = dfprodagroindustriales.join(dfagroindust, left_on='cultivo', right_on='idagro',how='inner').select(['llave','agroindustrial'
                                                     ,'epoca_agro','aplantillagro','areaprodagro','produccionagro','observacionagro'])
query = """
            select interview__id llave, epoca_emergente epoca, cultivos_emergentes cultivos,areaemergente area, produccemergente produccion,detalle_emergente detalle
            from "hq_dea_3a9df112-2351-459e-97a6-468d1cfaaf91"."Complemento$1"
"""
dfemergente = pl.DataFrame(pl.read_database_uri(query=query, uri=uri, engine='connectorx'))
dfemergente
# df = df.select(['interview__id','epoca_frutas'])
# query = """
#             select * from ws_dea.reusablecategoricaloptions where questionnaireid = 'a1b0a945-f180-4379-bd35-2faae3f2b739'
#             and categoriesid = '08f20ae9-440c-2054-0815-d28835790226'
# """
# df2 = pl.DataFrame(pl.read_database_uri(query=query, uri=uri, engine='connectorx'))
# df2 = df2.select(['value','text']).rename({'value':'idfruta','text':'Nfruta'})
# df = df.explode("epoca_frutas").cast({'epoca_frutas':pl.Int32})
# df2 = df.join(df2, left_on='epoca_frutas', right_on='idfruta', how='inner').groupby('interview__id').agg(
#     pl.col('Nfruta').str.concat(',')
# )
# print(df2)


In [ ]:
#conexion a sql sever, importante el inicio para reconocer la conexion 
urimssql = f'mssql+pyodbc://{usermsql}:{passmsql}@{hostmsql}/{dbnamemsql}'
query = """
        select FOLIO,DEPARTAMENTO, MUNICIPIO,NPRODUCTOR,CORR_SEG from FOLIO
"""
dfmuestraprod = pl.read_database_uri(query=query, uri=urimssql, engine='connectorx')#.cast({'FOLIO':pl.Float64})
urimssql = f'mssql+pyodbc://{usermsql}:{passmsql}@{hostmsql}/{dbnamemsql}'
query = """
        select anio, cast(folio as int) FOLIO, grano, departamento from SEC03
        inner join departamento on sec03.expldep = departamento.cod_dep
"""
dfmaiz = pl.read_database_uri(query=query, uri=urimssql, engine='connectorx')
dfprodmaiz = dfmuestraprod.join(dfmaiz, on='FOLIO', how='anti')
dfprodmaiz

In [ ]:
import pyodbc
#usando pyodbc para insertar el contenido del dfdepto en la base departamento
conn = pyodbc.connect('DRIVER={SQL Server};'
                       'SERVER=.\SQLDEV17;'
                       'DATABASE=TestCustomLogging;'
                       'UID=sa;'
                       'PWD=Mayo280522;'
)
cursor = conn.cursor()

for row in dfdepto.iter_rows():
    cursor.execute(
                    """
                      INSERT INTO Departamentos(IdDepto, Departamento)
                      VALUES(?,?)  
                    """,
                    row[0],
                    row[1]
    )
cursor.commit()

In [ ]:
nuevoDepto = pl.DataFrame({
            "iddepto": [15],
            "Departamento": ["Prueba depto"],
}
)
nuevoDepto = nuevoDepto.with_columns(nuevoDepto["iddepto"].cast(pl.Int32))
dfdepto = dfdepto.vstack(nuevoDepto)
uri = 'mssql+pyodbc://sa:Mayo280522@.\SQLDEV17/TestCustomLogging?driver=ODBC+Driver+17+for+SQL+Server'
uri2 = 'mssql+pyodbc://sa:Mayo280522@.\SQLDEV17/TestCustomLogging'
# dfdepto = dfdepto.rename({'Ndepartamento':'Departamento'})
# dfdepto = dfdepto.filter(pl.col('iddepto')<=14)
dfdepto
# dfdepto.write_database(table_name='Departamentos'
#                        ,connection=uri
#                        ,if_table_exists='append')



In [ ]:
query = """
        select iddepto from Departamentos
"""
cursor.execute(query)
resultado = cursor.fetchall()
dfdeptoLocal = pl.DataFrame(resultado)
#convirtiendo el resultado del cursor en tipo int32 y renombrando
dfdeptoLocal = dfdeptoLocal.with_columns(dfdeptoLocal['column_0'].apply(lambda x:x[0], return_dtype=pl.Int32).rename('iddepto')).select('iddepto')
#valido que filas del dataframe no existen(false) en la tabla sql
dfresultado = dfdepto.with_columns(dfdepto['iddepto'].is_in(dfdeptoLocal).alias('valida'))
#selecciono solo las filas que no existen en la tabla sql(false)
dfresultado = dfresultado.filter(pl.col('valida')== False).select('iddepto','Departamento')
#inserto solo las filas que no existian
dfresultado.write_database(table_name='Departamentos'
                           ,connection=uri
                           ,if_table_exists='append'
                        
)